In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor, HistGradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

In [2]:
root = '/content/drive/MyDrive/SnowPackPredictionChallenge'

In [4]:
# parse swe_data
df = pd.read_csv(root + '/feature_engineered_data.csv')
df

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude,precip,...,Rmin_roll3,Rmin_roll7,windspeed_roll3,windspeed_roll7,temp_range,snowfall,humidity_diff,day_of_year,humidity_temp_interaction,wind_humidity_interaction
0,Hannagan Meadows,33.65352,-109.30877,9027,0.888152,1991-01-08,279.40,33.65625,-109.28125,0.00,...,45.120000,43.608571,2.963333,4.830000,17.02,0.00,58.98,8,425.8356,155.1174
1,Hannagan Meadows,33.65352,-109.30877,9027,0.888152,1991-01-09,279.40,33.65625,-109.28125,0.35,...,37.123333,40.572857,3.130000,4.901429,15.22,0.00,70.67,9,446.6344,291.8671
2,Hannagan Meadows,33.65352,-109.30877,9027,0.888152,1991-01-10,281.94,33.65625,-109.28125,0.35,...,33.226667,40.642857,3.630000,4.955714,15.22,0.00,70.67,10,446.6344,291.8671
3,Hannagan Meadows,33.65352,-109.30877,9027,0.888152,1991-01-11,281.94,33.65625,-109.28125,0.00,...,28.913333,38.191429,4.096667,4.278571,17.62,0.00,71.92,11,403.4712,289.8376
4,Hannagan Meadows,33.65352,-109.30877,9027,0.888152,1991-01-12,281.94,33.65625,-109.28125,0.00,...,26.910000,35.060000,3.933333,3.545714,19.28,0.00,55.43,12,505.5216,201.7652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897995,Garver Creek,48.97523,-115.81915,4250,-0.927766,2016-12-27,73.66,48.96875,-115.84375,0.00,...,55.280000,45.525714,2.180000,2.528571,7.84,0.00,21.00,362,-85.4700,45.7800
1897996,Garver Creek,48.97523,-115.81915,4250,-0.927766,2016-12-28,81.28,48.96875,-115.84375,0.00,...,55.280000,47.964286,2.180000,2.441429,7.84,0.00,21.00,363,-85.4700,45.7800
1897997,Garver Creek,48.97523,-115.81915,4250,-0.927766,2016-12-29,83.82,48.96875,-115.84375,3.70,...,55.673333,50.571429,2.523333,2.501429,9.42,3.70,30.36,364,-17.9124,97.4556
1897998,Garver Creek,48.97523,-115.81915,4250,-0.927766,2016-12-30,83.82,48.96875,-115.84375,3.70,...,56.066667,53.178571,2.866667,2.561429,9.42,3.70,30.36,365,-17.9124,97.4556


In [5]:
# Extract features and target variable
features = ["Latitude", "Longitude", "Elevation", "Southness",
    "precip", "tmin", "tmax", "SPH", "SRAD", "Rmax", "Rmin", "windspeed",
    "SWE_lag1", "SWE_lag3", "SWE_lag7",
    "precip_lag1", "tmin_lag1", "tmax_lag1", "SPH_lag1",
    "SRAD_lag1", "Rmax_lag1", "Rmin_lag1", "windspeed_lag1",
    "SWE_roll3", "SWE_roll7", "precip_roll3", "tmin_roll3"]
target = "SWE"

In [6]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Drop missing values
df = df.dropna(subset=features + [target])

# Normalize the features using StandardScaler
scaler = StandardScaler()
df[features] = scaler.fit_transform(df[features])

# Splitting data into training and test sets
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.2, random_state=42)

X_train, y_train = train_df[features].values, train_df[target].values
X_val, y_val = val_df[features].values, val_df[target].values
X_test, y_test = test_df[features].values, test_df[target].values

# Reshape input for LSTM: (samples, time steps, features)
X_train = X_train.reshape((X_train.shape[0], 1, X_train.shape[1]))
X_val = X_val.reshape((X_val.shape[0], 1, X_val.shape[1]))
X_test = X_test.reshape((X_test.shape[0], 1, X_test.shape[1]))

# Build the LSTM model
model = Sequential([
    LSTM(64, return_sequences=True, input_shape=(1, X_train.shape[2])),
    Dropout(0.2),
    LSTM(32, return_sequences=False),
    Dropout(0.2),
    Dense(16, activation='relu'),
    Dense(1)  # Output layer for SWE prediction
])

# Compile the model
model.compile(optimizer='adam', loss='mse', metrics=['mae'])

# Train the model
history = model.fit(X_train, y_train, epochs=50, batch_size=32,
                    validation_data=(X_val, y_val), verbose=1)

# Evaluate the model on test data
test_loss, test_mae = model.evaluate(X_test, y_test, verbose=0)
print(f"\n✅ Advanced LSTM Model - Test MAE: {test_mae:.4f}")

# Make predictions
y_pred = model.predict(X_test).flatten()


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 178s 5ms/step - loss: 8573.7998 - mae: 25.8318 - val_loss: 100.5203 - val_mae: 5.7269
Epoch 2/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 173s 5ms/step - loss: 612.3018 - mae: 11.5969 - val_loss: 104.2173 - val_mae: 5.8034
Epoch 3/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 205s 5ms/step - loss: 455.2420 - mae: 10.3040 - val_loss: 96.6187 - val_mae: 4.2637
Epoch 4/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 198s 5ms/step - loss: 381.2953 - mae: 9.4962 - val_loss: 77.1846 - val_mae: 4.1808
Epoch 5/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 204s 5ms/step - loss: 331.6669 - mae: 8.8393 - val_loss: 91.1102 - val_mae: 4.6039
Epoch 6/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 200s 5ms/step - loss: 285.5135 - mae: 8.3095 - val_loss: 64.6181 - val_mae: 3.7638
Epoch 7/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 203s 5ms/step - loss: 251.2829 - mae: 7.8597 - val_loss: 50.6840 - val_mae: 3.9433
Epoch 8/50
37960/37960 ━━━━━━━━━━━━━━━━━━━━ 204s 5ms/step - loss: 223.2426 - mae: 7.3106 - val_loss: 72.5

In [7]:
# Compute evaluation metrics
from sklearn.metrics import mean_squared_error, r2_score
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)

print(f"\n📊 Advanced LSTM Model Performance:")
print(f"Root Mean Square Error (RMSE): {rmse:.4f}")
print(f"R² Score: {r2:.4f}")


📊 Advanced LSTM Model Performance:
Root Mean Square Error (RMSE): 22.5746
R² Score: 0.9922


In [11]:
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
r2 = r2_score(y_test, y_pred)
relative_bias = (np.sum(y_pred - y_test) / np.sum(y_test)) * 100
observed_mean = np.mean(y_test)
nse = 1 - (np.sum((y_pred - y_test) ** 2) / np.sum((y_test - observed_mean) ** 2))
# Compute Actual Error (Prediction - Observed)
actual_error = y_pred - y_test

evaluation_results = pd.DataFrame({
    "Metric": ["Nash-Sutcliffe Efficiency (NSE)", "Root Mean Square Error (RMSE)", "R² Score", "Relative Bias (%)", 'Prediction Error'],
    "Value": [nse, rmse, r2, relative_bias, actual_error]
})

# Display evaluation metrics
print("\n📊 Model Evaluation Metrics:")
evaluation_results


📊 Model Evaluation Metrics:


,Metric,Value
0,Nash-Sutcliffe Efficiency (NSE),0.992153
1,Root Mean Square Error (RMSE),22.574621
2,R² Score,0.992153
3,Relative Bias (%),5.466669
4,Prediction Error,"[2.6771697998046875, 0.2564582824707031, 0.280..."


In [12]:
predictions_df = pd.DataFrame({
    "Date": test_df["Date"],
    "Latitude": test_df["Latitude"],
    "Longitude": test_df["Longitude"],
    "SWE_actual": y_test,
    "SWE_predicted": y_pred
})
predictions_df.to_csv("prediction_LSTM.csv", index=False)
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df.to_csv("evaluation_LSTM.csv", index=False)

In [14]:
import joblib  # For saving scaler
from tensorflow.keras.models import load_model

# ✅ Save the trained LSTM model in the recommended format
model.save("lstm_model.keras")
print("✅ LSTM model saved successfully in Keras format.")

# ✅ Save the fitted scaler to ensure consistency during inference
joblib.dump(scaler, "scaler.pkl")
print("✅ Scaler saved successfully.")


✅ LSTM model saved successfully in Keras format.
✅ Scaler saved successfully.


In [ ]:
import pandas as pd
import numpy as np
import joblib  # For loading the scaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

# Load the trained model in Keras format
lstm_model = load_model("lstm_model.keras")
print("✅ LSTM model loaded successfully.")

# ✅ Step 2: Load new dataset
new_df = pd.read_csv(root + '/atl_test_data.csv')

# ✅ Step 3: Ensure the new dataset has the same features as used during training
features = ["Latitude", "Longitude", "Elevation", "Southness",
            "precip", "tmin", "tmax", "SPH", "SRAD", "Rmax", "Rmin", "windspeed",
            "SWE_lag1", "SWE_lag3", "SWE_lag7",
            "precip_lag1", "tmin_lag1", "tmax_lag1", "SPH_lag1",
            "SRAD_lag1", "Rmax_lag1", "Rmin_lag1", "windspeed_lag1",
            "SWE_roll3", "SWE_roll7", "precip_roll3", "tmin_roll3"]

# ✅ Step 4: Handle missing values
new_df = new_df.dropna(subset=features)

# ✅ Step 5: Extract feature data and scale it
X_new = new_df[features]
X_new_scaled = scaler.transform(X_new)  # Scale using the previously fitted scaler

# ✅ Step 6: Reshape for LSTM input (samples, time steps, features)
X_new_scaled = X_new_scaled.reshape((X_new_scaled.shape[0], 1, X_new_scaled.shape[1]))

# ✅ Step 7: Make predictions using LSTM
new_df["LSTM_SWE_Prediction"] = lstm_model.predict(X_new_scaled).flatten()

# ✅ Step 8: Save predictions to CSV
new_predictions_path = "lstm_new_data_predictions.csv"
new_df.to_csv(new_predictions_path, index=False)
print(f"✅ LSTM predictions saved to '{new_predictions_path}'")


In [16]:
import pandas as pd
import numpy as np
import joblib  # For loading the scaler
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [17]:
# Load the trained model in Keras format
lstm_model = load_model("lstm_model.keras")
print("✅ LSTM model loaded successfully.")

✅ LSTM model loaded successfully.


/usr/local/lib/python3.11/dist-packages/keras/src/saving/saving_lib.py:757: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 12 variables whereas the saved optimizer has 22 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [18]:
# ✅ Step 2: Load new dataset
new_df = pd.read_csv(root + '/atl_test_data.csv')
new_df

,lat,lon,Elevation,Southness,date,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,45.21875,-119.21875,5770,-0.124565,2017-01-01,3.575,100.00,71.78,0.0024,54.700,-3.09,-9.90,4.56
1,45.21875,-119.21875,5770,-0.124565,2017-01-02,0.600,99.67,67.66,0.0020,28.300,-5.27,-11.50,1.96
2,45.21875,-119.21875,5770,-0.124565,2017-01-03,0.000,82.28,55.65,0.0013,50.275,-7.88,-13.37,3.40
3,45.21875,-119.21875,5770,-0.124565,2017-01-04,0.000,96.78,44.43,0.0010,72.725,-9.18,-19.19,5.92
4,45.21875,-119.21875,5770,-0.124565,2017-01-05,0.000,81.10,27.47,0.0008,84.150,-5.78,-18.93,3.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10945,39.28125,-117.09375,8685,-0.168625,2019-12-27,0.000,67.91,54.06,0.0016,104.400,-6.22,-10.46,2.61
10946,39.28125,-117.09375,8685,-0.168625,2019-12-28,0.000,66.72,48.63,0.0016,104.225,-5.12,-10.48,3.26
10947,39.28125,-117.09375,8685,-0.168625,2019-12-29,1.650,66.96,41.88,0.0021,87.675,0.73,-6.87,2.23
10948,39.28125,-117.09375,8685,-0.168625,2019-12-30,0.900,74.63,60.05,0.0026,53.175,-1.20,-5.47,1.89


In [23]:
new_df = new_df.sort_values(by=['Latitude', 'Longitude', 'Date'])
new_df

,Latitude,Longitude,Elevation,Southness,Date,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
1095,36.53125,-106.34375,9249,-0.465294,2017-01-01,1.675,73.10,59.56,0.0026,102.500,-1.81,-5.81,3.56
1096,36.53125,-106.34375,9249,-0.465294,2017-01-02,0.325,63.30,56.28,0.0024,117.250,-2.03,-4.82,4.73
1097,36.53125,-106.34375,9249,-0.465294,2017-01-03,0.375,63.74,51.77,0.0023,114.175,-1.86,-5.83,4.16
1098,36.53125,-106.34375,9249,-0.465294,2017-01-04,11.500,71.43,55.95,0.0026,97.550,-1.11,-5.64,6.42
1099,36.53125,-106.34375,9249,-0.465294,2017-01-05,25.500,76.31,68.15,0.0033,94.425,-0.52,-3.40,10.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,45.21875,-119.21875,5770,-0.124565,2019-12-27,1.925,97.41,70.36,0.0027,36.175,-1.65,-6.50,4.06
1091,45.21875,-119.21875,5770,-0.124565,2019-12-28,0.000,91.07,61.84,0.0029,54.975,1.00,-4.57,2.56
1092,45.21875,-119.21875,5770,-0.124565,2019-12-29,0.000,86.97,52.21,0.0032,32.800,4.29,-2.85,1.59
1093,45.21875,-119.21875,5770,-0.124565,2019-12-30,0.000,99.07,82.74,0.0036,65.175,-0.06,-3.13,1.86


In [22]:
new_df = new_df.rename(columns={'lat': 'Latitude', 'lon': 'Longitude', 'date':'Date'})
new_df

,Latitude,Longitude,Elevation,Southness,Date,precip,Rmax,Rmin,SPH,SRAD,tmax,tmin,windspeed
0,45.21875,-119.21875,5770,-0.124565,2017-01-01,3.575,100.00,71.78,0.0024,54.700,-3.09,-9.90,4.56
1,45.21875,-119.21875,5770,-0.124565,2017-01-02,0.600,99.67,67.66,0.0020,28.300,-5.27,-11.50,1.96
2,45.21875,-119.21875,5770,-0.124565,2017-01-03,0.000,82.28,55.65,0.0013,50.275,-7.88,-13.37,3.40
3,45.21875,-119.21875,5770,-0.124565,2017-01-04,0.000,96.78,44.43,0.0010,72.725,-9.18,-19.19,5.92
4,45.21875,-119.21875,5770,-0.124565,2017-01-05,0.000,81.10,27.47,0.0008,84.150,-5.78,-18.93,3.59
...,...,...,...,...,...,...,...,...,...,...,...,...,...
10945,39.28125,-117.09375,8685,-0.168625,2019-12-27,0.000,67.91,54.06,0.0016,104.400,-6.22,-10.46,2.61
10946,39.28125,-117.09375,8685,-0.168625,2019-12-28,0.000,66.72,48.63,0.0016,104.225,-5.12,-10.48,3.26
10947,39.28125,-117.09375,8685,-0.168625,2019-12-29,1.650,66.96,41.88,0.0021,87.675,0.73,-6.87,2.23
10948,39.28125,-117.09375,8685,-0.168625,2019-12-30,0.900,74.63,60.05,0.0026,53.175,-1.20,-5.47,1.89


In [24]:
new_df.ffill(inplace=True)

In [27]:
dynamic_features = ['SWE', 'precip', 'tmin', 'tmax', 'SPH', 'SRAD', 'Rmax', 'Rmin', 'windspeed']

# Create Lagged Features (Past 1, 3, and 7 Days)
lag_days = [1, 3, 7]
for feature in dynamic_features:
    for lag in lag_days:
        new_df[f"{feature}_lag{lag}"] = df.groupby(['Latitude', 'Longitude'])[feature].shift(lag)

In [29]:
# Create Rolling Averages (3-day and 7-day windows)
rolling_windows = [3, 7]
for feature in dynamic_features:
    for window in rolling_windows:
        new_df[f"{feature}_roll{window}"] = df.groupby(['Latitude', 'Longitude'])[feature].rolling(window=window, min_periods=1).mean().reset_index(level=[0,1], drop=True)

In [30]:
new_df.dropna(inplace=True)

In [32]:
new_df['Date'] = pd.to_datetime(df['Date'])

In [35]:
new_df['temp_range'] = new_df['tmax'] - new_df['tmin']  # Daily temperature difference
new_df['snowfall'] = new_df['precip'] * (new_df['tmax'] < 0).astype(int)  # Assume snowfall if Tmax < 0°C
new_df['humidity_diff'] = new_df['Rmax'] - new_df['Rmin']  # Humidity difference
new_df['day_of_year'] = new_df['Date'].dt.dayofyear  # Extracts day of the year (1-365)

# Humidity and Wind Interaction
new_df['humidity_temp_interaction'] = new_df['humidity_diff'] * new_df['tmax']
new_df['wind_humidity_interaction'] = new_df['windspeed'] * new_df['humidity_diff']

In [37]:
new_df.to_csv("feature_engineered_atl_test_data.csv", index=False)

In [45]:
new_df.columns

Index(['Latitude', 'Longitude', 'Elevation', 'Southness', 'Date', 'precip',
       'Rmax', 'Rmin', 'SPH', 'SRAD', 'tmax', 'tmin', 'windspeed', 'SWE_lag1',
       'SWE_lag3', 'SWE_lag7', 'precip_lag1', 'precip_lag3', 'precip_lag7',
       'tmin_lag1', 'tmin_lag3', 'tmin_lag7', 'tmax_lag1', 'tmax_lag3',
       'tmax_lag7', 'SPH_lag1', 'SPH_lag3', 'SPH_lag7', 'SRAD_lag1',
       'SRAD_lag3', 'SRAD_lag7', 'Rmax_lag1', 'Rmax_lag3', 'Rmax_lag7',
       'Rmin_lag1', 'Rmin_lag3', 'Rmin_lag7', 'windspeed_lag1',
       'windspeed_lag3', 'windspeed_lag7', 'SWE_roll3', 'SWE_roll7',
       'precip_roll3', 'precip_roll7', 'tmin_roll3', 'tmin_roll7',
       'tmax_roll3', 'tmax_roll7', 'SPH_roll3', 'SPH_roll7', 'SRAD_roll3',
       'SRAD_roll7', 'Rmax_roll3', 'Rmax_roll7', 'Rmin_roll3', 'Rmin_roll7',
       'windspeed_roll3', 'windspeed_roll7', 'temp_range', 'snowfall',
       'humidity_diff', 'day_of_year', 'humidity_temp_interaction',
       'wind_humidity_interaction'],
      dtype='object')

In [46]:
# ✅ Step 3: Ensure the new dataset has the same features as used during training
features = ["Latitude", "Longitude", "Elevation", "Southness",
            "precip", "tmin", "tmax", "SPH", "SRAD", "Rmax", "Rmin", "windspeed",
            "SWE_lag1", "SWE_lag3", "SWE_lag7",
            "precip_lag1", "tmin_lag1", "tmax_lag1", "SPH_lag1",
            "SRAD_lag1", "Rmax_lag1", "Rmin_lag1", "windspeed_lag1",
            "SWE_roll3", "SWE_roll7", "precip_roll3", "tmin_roll3"]

# ✅ Step 4: Handle missing values
new_df = new_df.dropna(subset=features)

# ✅ Step 5: Extract feature data and scale it
X_new = new_df[features]
X_new_scaled = scaler.transform(X_new)  # Scale using the previously fitted scaler

# ✅ Step 6: Reshape for LSTM input (samples, time steps, features)
X_new_scaled = X_new_scaled.reshape((X_new_scaled.shape[0], 1, X_new_scaled.shape[1]))

# ✅ Step 7: Make predictions using LSTM
new_df["LSTM_SWE_Prediction"] = lstm_model.predict(X_new_scaled).flatten()

# ✅ Step 8: Save predictions to CSV
new_predictions_path = "lstm_new_data_predictions.csv"
new_df.to_csv(new_predictions_path, index=False)
print(f"✅ LSTM predictions saved to '{new_predictions_path}'")


342/342 ━━━━━━━━━━━━━━━━━━━━ 2s 4ms/step
✅ LSTM predictions saved to 'lstm_new_data_predictions.csv'


In [48]:
new_predictions = new_df["LSTM_SWE_Prediction"]
new_predictions


,LSTM_SWE_Prediction
1095,1161.543213
1096,1156.377930
1097,1156.118408
1098,464.522797
1099,460.453735
...,...
1090,1251.561279
1091,1265.615601
1092,1287.525391
1093,1318.847168


In [51]:
new_df

,Latitude,Longitude,Elevation,Southness,Date,precip,Rmax,Rmin,SPH,SRAD,...,Rmin_roll7,windspeed_roll3,windspeed_roll7,temp_range,snowfall,humidity_diff,day_of_year,humidity_temp_interaction,wind_humidity_interaction,LSTM_SWE_Prediction
1095,36.53125,-106.34375,9249,-0.465294,1994-01-07,1.675,73.10,59.56,0.0026,102.500,...,0.576192,-0.031477,-0.031477,4.00,1.675,13.54,7,-24.5074,48.2024,1161.543213
1096,36.53125,-106.34375,9249,-0.465294,1994-01-08,0.325,63.30,56.28,0.0024,117.250,...,0.576192,-0.031477,-0.031477,2.79,0.325,7.02,8,-14.2506,33.2046,1156.377930
1097,36.53125,-106.34375,9249,-0.465294,1994-01-09,0.375,63.74,51.77,0.0023,114.175,...,0.297772,0.125668,0.035871,3.97,0.375,11.97,9,-22.2642,49.7952,1156.118408
1098,36.53125,-106.34375,9249,-0.465294,1994-01-10,11.500,71.43,55.95,0.0026,97.550,...,0.019351,0.282813,0.103219,4.53,11.500,15.48,10,-17.1828,99.3816,464.522797
1099,36.53125,-106.34375,9249,-0.465294,1994-01-11,25.500,76.31,68.15,0.0033,94.425,...,-0.259069,0.439958,0.170567,2.88,25.500,8.16,11,-4.2432,84.2112,460.453735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,45.21875,-119.21875,5770,-0.124565,1994-01-02,1.925,97.41,70.36,0.0027,36.175,...,0.457464,-0.031477,-0.090135,4.85,1.925,27.05,2,-44.6325,109.8230,1251.561279
1091,45.21875,-119.21875,5770,-0.124565,1994-01-03,0.000,91.07,61.84,0.0029,54.975,...,0.623145,-0.031477,-0.148068,5.57,0.000,29.23,3,29.2300,74.8288,1265.615601
1092,45.21875,-119.21875,5770,-0.124565,1994-01-04,0.000,86.97,52.21,0.0032,32.800,...,0.576192,-0.031477,-0.031477,7.14,0.000,34.76,4,149.1204,55.2684,1287.525391
1093,45.21875,-119.21875,5770,-0.124565,1994-01-05,0.000,99.07,82.74,0.0036,65.175,...,0.576192,-0.031477,-0.031477,3.07,0.000,16.33,5,-0.9798,30.3738,1318.847168


In [52]:
df

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude,precip,...,Rmin_roll3,Rmin_roll7,windspeed_roll3,windspeed_roll7,temp_range,snowfall,humidity_diff,day_of_year,humidity_temp_interaction,wind_humidity_interaction
0,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-08,279.40,33.65625,-109.28125,-0.419515,...,45.120000,43.608571,2.963333,4.830000,17.02,0.00,58.98,8,425.8356,155.1174
1,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-09,279.40,33.65625,-109.28125,-0.363096,...,37.123333,40.572857,3.130000,4.901429,15.22,0.00,70.67,9,446.6344,291.8671
2,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-10,281.94,33.65625,-109.28125,-0.363096,...,33.226667,40.642857,3.630000,4.955714,15.22,0.00,70.67,10,446.6344,291.8671
3,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-11,281.94,33.65625,-109.28125,-0.419515,...,28.913333,38.191429,4.096667,4.278571,17.62,0.00,71.92,11,403.4712,289.8376
4,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-12,281.94,33.65625,-109.28125,-0.419515,...,26.910000,35.060000,3.933333,3.545714,19.28,0.00,55.43,12,505.5216,201.7652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897995,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-27,73.66,48.96875,-115.84375,-0.419515,...,55.280000,45.525714,2.180000,2.528571,7.84,0.00,21.00,362,-85.4700,45.7800
1897996,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-28,81.28,48.96875,-115.84375,-0.419515,...,55.280000,47.964286,2.180000,2.441429,7.84,0.00,21.00,363,-85.4700,45.7800
1897997,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-29,83.82,48.96875,-115.84375,0.176916,...,55.673333,50.571429,2.523333,2.501429,9.42,3.70,30.36,364,-17.9124,97.4556
1897998,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-30,83.82,48.96875,-115.84375,0.176916,...,56.066667,53.178571,2.866667,2.561429,9.42,3.70,30.36,365,-17.9124,97.4556


In [55]:
df["SWE"]

,SWE
0,279.40
1,279.40
2,281.94
3,281.94
4,281.94
...,...
1897995,73.66
1897996,81.28
1897997,83.82
1897998,83.82


In [ ]:
actual_df = df

predicted_df = new_df

# ✅ Ensure both DataFrames have the same merge keys
merge_keys = ["Date", "Latitude", "Longitude"]

# ✅ Merge actual and predicted SWE values (keep only matching rows)
merged_df = pd.merge(actual_df, predicted_df, on=merge_keys, how="inner")

# ✅ Rename columns for clarity
merged_df = merged_df.rename(columns={"SWE": "Actual_SWE", "LSTM_SWE_Prediction": "Predicted_SWE"})

# ✅ Verify row count before evaluation
print(f"✅ Number of matched rows: {len(merged_df)}")

# ✅ Ensure data is sorted for consistency
merged_df = merged_df.sort_values(by=merge_keys).reset_index(drop=True)

# Extract actual and predicted SWE values
y_actual = merged_df["Actual_SWE"].values
y_pred = merged_df["Predicted_SWE"].values

# ✅ Compute RMSE
rmse = np.sqrt(mean_squared_error(y_actual, y_pred))

# ✅ Compute R² Score
r2 = r2_score(y_actual, y_pred)

# ✅ Compute Nash-Sutcliffe Efficiency (NSE)
observed_mean = np.mean(y_actual)
nse = 1 - (np.sum((y_pred - y_actual) ** 2) / np.sum((y_actual - observed_mean) ** 2))

# ✅ Compute Relative Bias (%)
relative_bias = (np.sum(y_pred - y_actual) / np.sum(y_actual)) * 100

# ✅ Display Results
accuracy_metrics = pd.DataFrame({
    "Metric": ["Root Mean Square Error (RMSE)", "R² Score", "Nash-Sutcliffe Efficiency (NSE)", "Relative Bias (%)"],
    "Value": [rmse, r2, nse, relative_bias]
})

print("\n📊 Model Accuracy Metrics:")
print(accuracy_metrics)


In [56]:
actual_df = df
actual_df

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude,precip,...,Rmin_roll3,Rmin_roll7,windspeed_roll3,windspeed_roll7,temp_range,snowfall,humidity_diff,day_of_year,humidity_temp_interaction,wind_humidity_interaction
0,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-08,279.40,33.65625,-109.28125,-0.419515,...,45.120000,43.608571,2.963333,4.830000,17.02,0.00,58.98,8,425.8356,155.1174
1,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-09,279.40,33.65625,-109.28125,-0.363096,...,37.123333,40.572857,3.130000,4.901429,15.22,0.00,70.67,9,446.6344,291.8671
2,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-10,281.94,33.65625,-109.28125,-0.363096,...,33.226667,40.642857,3.630000,4.955714,15.22,0.00,70.67,10,446.6344,291.8671
3,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-11,281.94,33.65625,-109.28125,-0.419515,...,28.913333,38.191429,4.096667,4.278571,17.62,0.00,71.92,11,403.4712,289.8376
4,Hannagan Meadows,-2.447439,0.802992,0.813753,1.675491,1991-01-12,281.94,33.65625,-109.28125,-0.419515,...,26.910000,35.060000,3.933333,3.545714,19.28,0.00,55.43,12,505.5216,201.7652
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1897995,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-27,73.66,48.96875,-115.84375,-0.419515,...,55.280000,45.525714,2.180000,2.528571,7.84,0.00,21.00,362,-85.4700,45.7800
1897996,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-28,81.28,48.96875,-115.84375,-0.419515,...,55.280000,47.964286,2.180000,2.441429,7.84,0.00,21.00,363,-85.4700,45.7800
1897997,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-29,83.82,48.96875,-115.84375,0.176916,...,55.673333,50.571429,2.523333,2.501429,9.42,3.70,30.36,364,-17.9124,97.4556
1897998,Garver Creek,1.728488,-0.473609,-1.616377,-1.901915,2016-12-30,83.82,48.96875,-115.84375,0.176916,...,56.066667,53.178571,2.866667,2.561429,9.42,3.70,30.36,365,-17.9124,97.4556


In [58]:
predicted_df = new_df
predicted_df

,Latitude,Longitude,Elevation,Southness,Date,precip,Rmax,Rmin,SPH,SRAD,...,Rmin_roll7,windspeed_roll3,windspeed_roll7,temp_range,snowfall,humidity_diff,day_of_year,humidity_temp_interaction,wind_humidity_interaction,LSTM_SWE_Prediction
1095,36.53125,-106.34375,9249,-0.465294,1994-01-07,1.675,73.10,59.56,0.0026,102.500,...,0.576192,-0.031477,-0.031477,4.00,1.675,13.54,7,-24.5074,48.2024,1161.543213
1096,36.53125,-106.34375,9249,-0.465294,1994-01-08,0.325,63.30,56.28,0.0024,117.250,...,0.576192,-0.031477,-0.031477,2.79,0.325,7.02,8,-14.2506,33.2046,1156.377930
1097,36.53125,-106.34375,9249,-0.465294,1994-01-09,0.375,63.74,51.77,0.0023,114.175,...,0.297772,0.125668,0.035871,3.97,0.375,11.97,9,-22.2642,49.7952,1156.118408
1098,36.53125,-106.34375,9249,-0.465294,1994-01-10,11.500,71.43,55.95,0.0026,97.550,...,0.019351,0.282813,0.103219,4.53,11.500,15.48,10,-17.1828,99.3816,464.522797
1099,36.53125,-106.34375,9249,-0.465294,1994-01-11,25.500,76.31,68.15,0.0033,94.425,...,-0.259069,0.439958,0.170567,2.88,25.500,8.16,11,-4.2432,84.2112,460.453735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,45.21875,-119.21875,5770,-0.124565,1994-01-02,1.925,97.41,70.36,0.0027,36.175,...,0.457464,-0.031477,-0.090135,4.85,1.925,27.05,2,-44.6325,109.8230,1251.561279
1091,45.21875,-119.21875,5770,-0.124565,1994-01-03,0.000,91.07,61.84,0.0029,54.975,...,0.623145,-0.031477,-0.148068,5.57,0.000,29.23,3,29.2300,74.8288,1265.615601
1092,45.21875,-119.21875,5770,-0.124565,1994-01-04,0.000,86.97,52.21,0.0032,32.800,...,0.576192,-0.031477,-0.031477,7.14,0.000,34.76,4,149.1204,55.2684,1287.525391
1093,45.21875,-119.21875,5770,-0.124565,1994-01-05,0.000,99.07,82.74,0.0036,65.175,...,0.576192,-0.031477,-0.031477,3.07,0.000,16.33,5,-0.9798,30.3738,1318.847168


In [62]:
# ✅ Ensure both DataFrames have the same merge keys
merge_keys = ["Latitude", "Longitude"]

# ✅ Merge actual and predicted SWE values (keep only matching rows)
merged_df = pd.merge(actual_df, predicted_df, on=merge_keys, how="inner")
merged_df

,Station,Latitude,Longitude,Elevation_x,Southness_x,Date_x,SWE,nearest_md_latitude,nearest_md_longitude,precip_x,...,Rmin_roll7_y,windspeed_roll3_y,windspeed_roll7_y,temp_range_y,snowfall_y,humidity_diff_y,day_of_year_y,humidity_temp_interaction_y,wind_humidity_interaction_y,LSTM_SWE_Prediction


In [60]:
df['Date'] = pd.to_datetime(df['Date'])

In [80]:
# Extract actual and predicted values
y_actual = merged_swe["SWE"].values
y_pred = merged_df["LSTM_SWE_Prediction"].values

# Compute RMSE
rmse = np.sqrt(mean_squared_error(y_actual, y_pred))

# Compute R² Score
r2 = r2_score(y_actual, y_pred)

# Compute Nash-Sutcliffe Efficiency (NSE)
observed_mean = np.mean(y_actual)
nse = 1 - (np.sum((y_pred - y_actual) ** 2) / np.sum((y_actual - observed_mean) ** 2))

# Compute Relative Bias (%)
relative_bias = (np.sum(y_pred - y_actual) / np.sum(y_actual)) * 100

# Display Results
accuracy_metrics = pd.DataFrame({
    "Metric": ["Root Mean Square Error (RMSE)", "R² Score", "Nash-Sutcliffe Efficiency (NSE)", "Relative Bias (%)"],
    "Value": [rmse, r2, nse, relative_bias]
})

print("\n📊 Model Accuracy Metrics:")
print(accuracy_metrics)


ValueError: Found input variables with inconsistent numbers of samples: [1899400, 10936]

In [67]:
merged_swe = pd.read_csv(root + '/merged_swe.csv')
merged_swe

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76
...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94


In [68]:
from scipy.spatial import cKDTree

# extract spatial coordinates
swe_coords = merged_swe[['Latitude', 'Longitude']].values
md_coords = new_df[['Latitude', 'Longitude']].drop_duplicates().values

# build KD-Tree using meteorological grid points
tree = cKDTree(md_coords)

# find the nearest meteorological grid point for each SWE station
distances, indices = tree.query(swe_coords, k=1)

# map each station to its closest meteorological grid point
merged_swe['nearest_md_latitude'] = md_coords[indices, 0]
merged_swe['nearest_md_longitude'] = md_coords[indices, 1]

In [69]:
merged_swe

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76,45.21875,-119.21875
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76,45.21875,-119.21875
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76,45.21875,-119.21875
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76,45.21875,-119.21875
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76,45.21875,-119.21875
...,...,...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48,36.53125,-106.34375
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48,36.53125,-106.34375
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48,36.53125,-106.34375
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94,36.53125,-106.34375


In [4]:
merged_df = new_df.rename(columns={'Latitude': 'nearest_md_latitude', 'Longitude': 'nearest_md_longitude'})

final_data = pd.merge(
    merged_swe, merged_df,
    on=['Date', 'nearest_md_latitude', 'nearest_md_longitude'],
    how='left'
)

NameError: name 'new_df' is not defined

In [71]:
merged_swe['Date'] = pd.to_datetime(merged_swe['Date'])

In [82]:
merged_df

,nearest_md_latitude,nearest_md_longitude,Elevation,Southness,Date,precip,Rmax,Rmin,SPH,SRAD,...,Rmin_roll7,windspeed_roll3,windspeed_roll7,temp_range,snowfall,humidity_diff,day_of_year,humidity_temp_interaction,wind_humidity_interaction,LSTM_SWE_Prediction
1095,36.53125,-106.34375,9249,-0.465294,1994-01-07,1.675,73.10,59.56,0.0026,102.500,...,0.576192,-0.031477,-0.031477,4.00,1.675,13.54,7,-24.5074,48.2024,1161.543213
1096,36.53125,-106.34375,9249,-0.465294,1994-01-08,0.325,63.30,56.28,0.0024,117.250,...,0.576192,-0.031477,-0.031477,2.79,0.325,7.02,8,-14.2506,33.2046,1156.377930
1097,36.53125,-106.34375,9249,-0.465294,1994-01-09,0.375,63.74,51.77,0.0023,114.175,...,0.297772,0.125668,0.035871,3.97,0.375,11.97,9,-22.2642,49.7952,1156.118408
1098,36.53125,-106.34375,9249,-0.465294,1994-01-10,11.500,71.43,55.95,0.0026,97.550,...,0.019351,0.282813,0.103219,4.53,11.500,15.48,10,-17.1828,99.3816,464.522797
1099,36.53125,-106.34375,9249,-0.465294,1994-01-11,25.500,76.31,68.15,0.0033,94.425,...,-0.259069,0.439958,0.170567,2.88,25.500,8.16,11,-4.2432,84.2112,460.453735
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1090,45.21875,-119.21875,5770,-0.124565,1994-01-02,1.925,97.41,70.36,0.0027,36.175,...,0.457464,-0.031477,-0.090135,4.85,1.925,27.05,2,-44.6325,109.8230,1251.561279
1091,45.21875,-119.21875,5770,-0.124565,1994-01-03,0.000,91.07,61.84,0.0029,54.975,...,0.623145,-0.031477,-0.148068,5.57,0.000,29.23,3,29.2300,74.8288,1265.615601
1092,45.21875,-119.21875,5770,-0.124565,1994-01-04,0.000,86.97,52.21,0.0032,32.800,...,0.576192,-0.031477,-0.031477,7.14,0.000,34.76,4,149.1204,55.2684,1287.525391
1093,45.21875,-119.21875,5770,-0.124565,1994-01-05,0.000,99.07,82.74,0.0036,65.175,...,0.576192,-0.031477,-0.031477,3.07,0.000,16.33,5,-0.9798,30.3738,1318.847168


In [84]:
merged_swe

,Station,Latitude,Longitude,Elevation,Southness,Date,SWE,nearest_md_latitude,nearest_md_longitude
0,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-01,111.76,45.21875,-119.21875
1,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-02,111.76,45.21875,-119.21875
2,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-03,111.76,45.21875,-119.21875
3,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-04,111.76,45.21875,-119.21875
4,Arbuckle Mtn,45.19085,-119.25392,5770,-0.124565,1991-01-05,111.76,45.21875,-119.21875
...,...,...,...,...,...,...,...,...,...
1899395,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-27,30.48,36.53125,-106.34375
1899396,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-28,30.48,36.53125,-106.34375
1899397,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-29,30.48,36.53125,-106.34375
1899398,Workman Creek,33.81259,-110.91852,7032,0.922266,2016-12-30,27.94,36.53125,-106.34375


In [85]:
merged_df = new_df.rename(columns={'nearest_md_latitude': 'Latitude', 'nearest_md_longitude': 'Longitude'})

In [3]:
# ✅ Ensure both DataFrames have the same merge keys
merge_keys = ["Latitude", "Longitude"]

# ✅ Merge actual and predicted SWE values (keep only matching rows)
merged_df1 = pd.merge(merged_df, new_df, on=merge_keys, how="inner")
merged_df1

NameError: name 'merged_df' is not defined